In [1]:
import numpy as np
import cvxpy as cp
import numpy as np
import pandas as pd
import time
from scipy.optimize import minimize
import random
from tqdm.notebook import tqdm

# 行動分類のラベル
large = ["必需行動", "拘束行動", "自由行動", "その他"] # 大分類
 
middle = ["睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事関連", "学業", 
  "家事", "通勤", "通学", "社会参加", "会話・交際", "レジャー活動", 
  "マスメディア接触", "休息", "その他・不明"]
 # 中分類
 
small = ["睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい", "授業", "学校外の学習", 
  "炊事・掃除・洗濯", "買い物", "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加", "会話・交際", 
  "スポーツ", "行楽・散策", "趣味・娯楽・教養 (インターネット除く)", "趣味・娯楽・教養のインターネット (動画除く)", 
  "インターネット動画", "テレビ", "録画番組・DVD", "ラジオ", "新聞", "雑誌・マンガ・本", "音楽", "休息", "その他", "不明"]
 # 小分類

def calc_list_diff(list1:list, list2:list):
    # リストの長さが異なる場合のエラーチェック
    if len(list1) != len(list2):
        raise ValueError("リストの長さが異なります。")
    
    # 各要素の差分を計算して新しいリストを生成
    difference = [a - b for a, b in zip(list1, list2)]
    return difference

# 遷移確率の計算
def calc_transition_rates(beta_i:np.ndarray,time_list:list,time_table:pd.DataFrame):
    transition_rates = []
    for i in range(len(time_list)):
        time_t      = time_list[i]
        time_t1     = time_list[(i+1)%(24*4)]
        y_t     = np.array(sort_df_by_list(time_table.query("Time == @time_t"),"Activity",middle)["Rate"])
        y_t1    = np.array(sort_df_by_list(time_table.query("Time == @time_t1"),"Activity",middle)["Rate"])

        # 変数の定義
        no_of_elem = len(middle)
        # 変数の定義
        a = cp.Variable((no_of_elem, no_of_elem), nonneg=True)  # 非負制約を持つ変数行列

        # パラメータの設定
        delta = np.eye(no_of_elem)  # クロネッカーのデルタ
        beta_i = np.array(np.ones(no_of_elem))

        # 目的関数の定義（要素ごとの積を使用）
        objective = cp.Minimize(cp.sum(cp.multiply(1 - beta_i * delta, cp.square(a))))

        # 制約条件の定義
        constraints = [
            cp.sum(a, axis=0) == 1,  # 制約 (6b)
            a >= 0,  # 制約 (6c)
        ]

        # 正規化
        y_t = y_t / np.sum(y_t)
        y_t1 = y_t1 / np.sum(y_t1)

        # 制約 (6d)
        constraints.append(y_t1 - a @ y_t == 0) # (6d)

        # 問題の定義と解法
        problem = cp.Problem(objective, constraints)
        problem.solve()
        # 結果の格納
        transition_rates.append(a.value)

    return transition_rates

def get_p_i(initial_rate:np.ndarray,transition_rates:list):
    initial_rate = initial_rate / np.sum(initial_rate)# 正規化
    p_i_list = [] # 算出した各行動種別のpiを保持するリスト
    for act_type in range(len(middle)): # 各行動種別ごとのloop

        # 当該行動タイプのへの遷移確率を0に変更するために遷移確率の一部を編集する。
        a_s = transition_rates.copy() 
        # 行為数分の1ベクトルを生成
        n = len(middle)
        one_vector = np.ones(n)
        modified_transition_rates = []

        for A in a_s: # 遷移確率の各時間ごとのloop
            A_act = A.copy()
            A_act[act_type, :] = 0  # 行：該当行動種別 を全て 0 に置き換える
            modified_transition_rates.append(A_act)

        # 初期行為者率ベクトルの加工
        modified_initial_rate = initial_rate.copy()
        modified_initial_rate[act_type] = 0

        # 最終的な行為者率ベクトルの計算
        final_rate = modified_initial_rate

        for A_i in modified_transition_rates:
            final_rate = A_i @ final_rate

        # 行為 i を一度も行わない確率の計算
        p_not_i = one_vector @ final_rate

        p_i = 1-p_not_i
        p_i_list.append(p_i)
    
    return np.array(p_i_list)

def roulette_wheel_selection(weights):
    """
    Performs roulette wheel selection on a list of weights.
    
    Args:
    weights (list of float): The weights or probabilities for each item.
    
    Returns:
    int: The index of the selected item.
    """
    # Calculate the cumulative sum of weights
    cumulative_sum = [sum(weights[:i+1]) for i in range(len(weights))]
    total_sum = cumulative_sum[-1]
    
    # Generate a random number in the range [0, total_sum)
    random_num = random.uniform(0, total_sum)
    
    # Find the index where the random number would fit in the cumulative sum
    for i, cum_sum in enumerate(cumulative_sum):
        if random_num < cum_sum:
            return i

def sort_df_by_list(df:pd.DataFrame,sort_column_by:str,sort_key_by:list):
    df = df.copy()  # 明示的にDataFrameのコピーを作成
    df.loc[:,"sort_order"] = df[sort_column_by].map({name: i for i,name in enumerate(sort_key_by)})
    df_sorted = df.sort_values('sort_order').drop('sort_order', axis=1)
    return df_sorted

(CVXPY) Sep 01 03:42:58 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: 指定されたモジュールが見つかりません。')


In [8]:
"""国民生活時間調査のデータからの成型"""
# daily
daily_df = pd.read_csv('2020_4shihyo_all.csv')
daily_df = daily_df.set_axis(["Day","Group","Activity","Rate","AvgTimeAct","AvgTimeAll","StdDevAll"],axis=1)
daily_df["Rate"] = daily_df["Rate"]/100
# daily_table = daily_df.query("Day == '平日' and Group == '男１０代'").query(f"Activity in @middle").filter(items=["Activity","Rate"])
daily_table = daily_df.query("Day == '平日'").query(f"Activity in @middle").filter(items=["Activity","Rate"])
# time
time_df = pd.read_csv('2020_jikoku_all.csv')
time_df = time_df.set_axis(["Day","Group","Activity","Time","Rate"],axis=1)
time_df["Rate"] = time_df["Rate"]/100
# time_table = time_df.query("Day == '平日' and Group == '男１０代'").query(f"Activity in @middle").filter(items=["Activity","Time","Rate"])
time_table = time_df.query("Day == '平日'").query(f"Activity in @middle").filter(items=["Activity","Time","Rate"])
# get time list
time_list = list(time_table["Time"].unique())

orig_P_i = daily_table["Rate"].to_list() # オリジナルの一日の行為者率

In [4]:
time_table.to_csv("C:/Users/tora2/downloads/time_table.csv")

,Unnamed: 0,0:00,1:00,2:00,3:00,4:00,5:00,6:00,7:00,8:00,...,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00
0,Day 1,身のまわりの用事,休息,レジャー活動,療養・静養,その他・不明,仕事関連,仕事関連,睡眠,レジャー活動,...,その他・不明,休息,マスメディア接触,休息,食事,休息,食事,家事,通勤,学業
1,Day 2,身のまわりの用事,レジャー活動,身のまわりの用事,社会参加,家事,レジャー活動,学業,身のまわりの用事,休息,...,通学,療養・静養,会話・交際,通学,社会参加,会話・交際,通勤,レジャー活動,食事,学業
2,Day 3,学業,マスメディア接触,休息,食事,学業,その他・不明,通学,休息,身のまわりの用事,...,レジャー活動,その他・不明,学業,通勤,食事,仕事関連,学業,その他・不明,学業,食事
3,Day 4,休息,休息,レジャー活動,仕事関連,睡眠,マスメディア接触,社会参加,家事,仕事関連,...,仕事関連,家事,会話・交際,睡眠,学業,会話・交際,療養・静養,仕事関連,仕事関連,学業
4,Day 5,マスメディア接触,家事,仕事関連,療養・静養,休息,家事,会話・交際,仕事関連,睡眠,...,学業,会話・交際,食事,通学,マスメディア接触,学業,通学,身のまわりの用事,レジャー活動,社会参加
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Day 996,会話・交際,療養・静養,会話・交際,身のまわりの用事,休息,会話・交際,家事,学業,仕事関連,...,社会参加,休息,休息,社会参加,睡眠,家事,その他・不明,通学,マスメディア接触,休息
996,Day 997,療養・静養,家事,通学,休息,療養・静養,通勤,マスメディア接触,通学,社会参加,...,療養・静養,会話・交際,療養・静養,レジャー活動,家事,家事,会話・交際,身のまわりの用事,食事,その他・不明
997,Day 998,会話・交際,休息,会話・交際,食事,レジャー活動,社会参加,通勤,睡眠,家事,...,身のまわりの用事,学業,マスメディア接触,睡眠,レジャー活動,通勤,レジャー活動,身のまわりの用事,睡眠,療養・静養
998,Day 999,学業,学業,通勤,社会参加,仕事関連,会話・交際,その他・不明,マスメディア接触,睡眠,...,マスメディア接触,休息,通学,その他・不明,通学,その他・不明,家事,休息,学業,身のまわりの用事


In [14]:
"""SA法によるβのパラメータフィッティング"""
def objective(beta_arr:np.ndarray,time_list:list,time_table:pd.DataFrame,daily_table:pd.DataFrame,act_label:list): # sa法の目的関数
    transition_rates    = calc_transition_rates(beta_arr,time_list,time_table) # 遷移確率の計算
    time_table_0_00     = sort_df_by_list(time_table.query("Time == @time_list[0]"),"Activity",act_label) # 時刻0:00における行為者率の元データ
    initial_rate        = np.array(time_table_0_00["Rate"].to_list()) # 時刻0:00における行為者率（元データ）
    calculated_p_i      = get_p_i(initial_rate,transition_rates) # 遷移確率から算出されたp_i
    origin_p_i          = sort_df_by_list(daily_table,"Activity",middle)["Rate"].to_list() # 一日の行為者率の元データ
    diff_1              = [diff for diff in calc_list_diff(list(calculated_p_i),origin_p_i)] # 各p_iでのオリジナルデータのとの差分
    diff_2              = sum(diff**2 for diff in diff_1) # 格差分の二乗和
    return diff_1,diff_2


""" simulated annealing """
# sa法のパラメータ
N_ITERATIONS = 10
STEP_SIZE = 0.1
TEMP = 100.0
# 評価関数のパラメータ
ACTION_LABEL = middle
INITIAL_BETA = 0.5
BOUNDS = len(ACTION_LABEL) # βの要素数は行動種別の数と一致する。

initial_beta = np.array(np.ones(BOUNDS)*INITIAL_BETA) # 初期解生成
best_eval1, best_eval2 = objective(initial_beta,time_list,time_table,daily_table,ACTION_LABEL) # 初期評価値を計算
current, current_eval1, current_eval2 = initial_beta, best_eval1, best_eval2 # 現状解と現状評価に代入

with tqdm(total=len(range(N_ITERATIONS)), desc="Processing", bar_format='{percentage:3.0f}% |{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}{postfix}] {desc}') as pbar:
    for i in range(N_ITERATIONS):
        # 新しい候補を生成
        # differents = np.random.uniform(-STEP_SIZE,STEP_SIZE,BOUNDS)
        # differents_updated = []
        # for diff, eval1 in zip(list(differents),current_eval1):
        #     new_diff = 0
        #     if (eval1 > 0): # オリジナルの行為者率よりも高い場合→遷移しすぎ→βを小さく(遷移のしやすさを抑制)
        #         new_diff = abs(diff)*(-1)
        #     elif (eval1 < 0): # オリジナルの行為者率よりも低い場合→遷移しなさすぎ→βを大きく(遷移のしやすさを促進)
        #         new_diff = abs(diff)
        #     else:
        #         new_diff = diff
        #     differents_updated.append(new_diff)
        candidate = current + np.array(np.random.uniform(-STEP_SIZE,STEP_SIZE,BOUNDS))#differents_updated)

        # βは0<=β<=1
        candidate_checked = []
        for c in list(candidate):
            if c > 1:
                candidate_checked.append(1)
            elif c < 0:
                candidate_checked.append(0)
            else:
                candidate_checked.append(c)
        candidate_checked = np.array(candidate_checked)


        # 新しい評価値を計算
        candidate_eval1, candidate_eval2 = objective(candidate_checked,time_list,time_table,daily_table,ACTION_LABEL)
        # 温度の減衰
        t = TEMP / float(i + 1)
        # 新しい解を受け入れるかの確率を計算
        if candidate_eval2 < current_eval2 or np.exp((current_eval2 - candidate_eval2) / t) > np.random.rand():
            current, current_eval2 = candidate, candidate_eval2
        # ベスト解を更新
        if candidate_eval2 < best_eval2:
            best, best_eval1, best_eval2 = candidate, candidate_eval1, candidate_eval2
            print(f"Iteration {i+1}, Best Evaluation: {best_eval2}")
        pbar.set_description_str(f"Iteration {i+1}, Best Evaluation: {best_eval2}")
        pbar.update(1)

  0% |          | 0/10 [00:00<?, ?it/s] Processing

In [17]:
current

array([0.58530375, 0.52601123, 0.42074461, 0.43369628, 0.36465379,
       0.39106651, 0.619314  , 0.70748569, 0.62336047, 0.63439629,
       0.64464465, 0.60312289, 0.23633318, 0.27782043, 0.44061768])

In [58]:
"""算出した遷移確率から一個票を生成"""
transition_rates = calc_transition_rates(current,time_list,time_table)
# 推定したβをもとに遷移確率を算出
no_of_pattern = 1000

pattern_arr = []

for i in range(no_of_pattern):
    # 初期行動の規定
    time_table_0 = sort_df_by_list(time_table.query("Time == '0:00'"),"Activity",middle) # 0:00における行為者率を抽出
    action_idx = roulette_wheel_selection(time_table_0["Rate"].to_list()) # その時刻における行為者率を重みとしてルーレット選択
    Actions =[] # 選択した行為種別の記録
    Actions.append(action_idx)
    for t in range(4*24-1): # 15min間隔で24時間分繰り返す。
        transition_r_t = transition_rates[t] # 時刻tの遷移確率行列を抽出
        action_idx = roulette_wheel_selection(list(transition_r_t[action_idx,:])) # 遷移確率をもとにt+1における行動種別を決定
        Actions.append(action_idx) # 行動種別履歴に追加
    acts = daily_table["Activity"].to_list()
    actMaster = dict(zip(list(range(len(acts))),acts))
    pattern_arr.append([actMaster[a] for a in Actions])

In [46]:
pd.set_option("display.max_columns",None)

In [57]:
df = pd.read_csv("C:/Users/tora2/downloads/persona_schedule.csv")
df

,曜日,時間,行為種別,詳細
0,月曜日,6:00 - 6:30,身のまわりの用事,目覚ましが鳴り、少し寝ぼけながら起床。今日は新しいプロジェクトのキックオフの日。軽いストレッ...
1,月曜日,6:30 - 7:00,朝食,子供たちを起こして、簡単な朝食を準備。目玉焼きとトースト、そしてフルーツ。夫と今日のスケジュ...
2,月曜日,7:00 - 8:30,通勤,電車に乗って通勤。途中でメールをチェックし、プロジェクトに関連する資料を再確認。少し緊張しつ...
3,月曜日,8:30 - 12:00,仕事関連,新プロジェクトのキックオフミーティング。クライアントからの要求が増えていて、少し不安を感じる...
4,月曜日,12:00 - 12:30,昼食,急いでランチを取る。サンドイッチを片手に、サーバー復旧作業を指示。なんとか午後のミーティング...
5,月曜日,12:30 - 16:00,仕事関連,午後はクライアントとの追加打ち合わせ。トラブルの影響で予定が押しているため、集中して対応。ト...
6,月曜日,16:00 - 17:30,帰宅,電車に乗って帰宅。今日は早めに帰り、家族と夕食の時間を確保する。帰宅途中、子供が学校で風邪を...
7,月曜日,17:30 - 18:00,家事・夕食準備,子供の様子を見つつ、簡単な夕食を準備。夫が薬を買って帰宅。みんなで夕食を取るが、子供の体調が...
8,月曜日,18:00 - 19:30,家族との時間,子供を寝かしつける。夫と今日の出来事を話し合い、明日のスケジュールを再確認。家事を手伝っても...
9,月曜日,19:30 - 21:00,リラックス,今日は疲れたので、早めにベッドに入る。読書をしながら、明日の業務の準備を少しだけ進める。


In [42]:
# 各時間帯ごとの行動種別のカウント
activity_counts = df.apply(pd.Series.value_counts).fillna(0)

# 各時間帯ごとの割合を計算
activity_ratios = activity_counts.div(activity_counts.sum(axis=0), axis=1)
calc_df = activity_ratios.reset_index().melt(id_vars=['index'], var_name='Time', value_name='Rate')
calc_df.rename(columns={'index': 'Activity'}, inplace=True)

In [43]:
calc_df["Time"] = pd.to_datetime(calc_df["Time"],format="%H:%M").dt.time
Calc_df = sort_df_by_list(calc_df,"Activity",middle)
Time_table = time_table.copy()
Time_table["Time"] = pd.to_datetime(Time_table["Time"],format="%H:%M").dt.time

In [44]:
import pandas as pd
import plotly.graph_objects as go

# 各アクティビティに対して色を指定
color_map = {
    '睡眠': 'blue',
    'その他・不明': 'red',
    'マスメディア接触': 'green',
    'レジャー活動': 'purple',
    '仕事関連': 'orange',
    '休息': 'pink',
    '会話・交流': 'cyan',
    '学業': 'brown',
    '家事': 'yellow',
    '療養・静養': 'black',
    '社会参加': 'magenta',
    '身のまわりの用事': 'lightblue',
    '通勤': 'darkgreen',
    '通学': 'lightgreen',
    '食事': 'gray'
}

# Plotlyで折れ線グラフを作成
fig = go.Figure()

# time_tableの実線グラフ
for activity in Time_table['Activity'].unique():
    filtered_df = Time_table[Time_table['Activity'] == activity]
    fig.add_trace(go.Scatter(
        x=filtered_df['Time'], 
        y=filtered_df['Rate'], 
        mode='lines', 
        name=f'{activity} (実線)', 
        line=dict(dash='solid', color=color_map.get(activity))
    ))

# calc_dfの点線グラフ
for activity in calc_df['Activity'].unique():
    filtered_df = calc_df[calc_df['Activity'] == activity]
    fig.add_trace(go.Scatter(
        x=filtered_df['Time'], 
        y=filtered_df['Rate'], 
        mode='lines', 
        name=f'{activity} (点線)', 
        line=dict(dash='dot', color=color_map.get(activity))
    ))

fig.update_layout(
    title="Activityごとの時間推移",
    xaxis_title="時間",
    yaxis_title="割合",
    legend_title="活動",
    template="plotly_dark"
)

fig.show()


In [258]:
fig.write_html(r"C:\Users\tora2\Downloads\graph2.html")

In [1]:
Time_table

NameError: name 'Time_table' is not defined